In [1]:
import datetime as dt
import pandas as pd
import os
import cisco_fiscal_calendar as cfc
import compass_readouts as cr

C:\Users\phsheari\Notebooks\compass_utils\compass_readouts.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  step_one['Readout_Date'] = step_one['Readout_Date'].apply(lambda x: pd.to_datetime(x))
C:\Users\phsheari\Notebooks\compass_utils\compass_readouts.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  guidf['GUID'] = guidf['GUID'].apply(stringify_element)


In [2]:
pd.options.display.float_format = '{:,.2f}'.format

In [3]:
weeks = 9, 10, None

In [4]:
nonecount = 0

In [5]:
for item in weeks:
    if item == None:
        nonecount +=1
    
weeks_length = len(weeks) - nonecount

In [6]:
weeks_length

2

In [7]:
weekA = weeks[0]
weekB = weeks[1]
weekC = weeks[2]

In [8]:
weekA, weekB, weekC

(9, 10, None)

In [9]:
if weekC == None:
    endfilename = f'FINBI_APJC_Bookings_FY22Q3_W{weekA}_W{weekB}.xlsx'
else:
    endfilename = f'FINBI_APJC_Bookings_FY22Q3_W{weekA}_W{weekB}_W{weekC}.xlsx'
print(endfilename)

FINBI_APJC_Bookings_FY22Q3_W9_W10.xlsx


In [10]:
def get_reporting_week(finbi_fiscal_week_id):
    return cal.loc[cal['finbi_fiscal_week_id']==finbi_fiscal_week_id, 'week_of_fiscalquarter'].values[0]

In [11]:
def convert_nan_to_int(dataframe_cell):
        if pd.isnull(dataframe_cell):
            return pd.NA
        else:
            try:
                return int(dataframe_cell)
            except:
                return pd.NA

In [12]:
def convert_nan_to_int_to_str(dataframe_cell):
        if pd.isnull(dataframe_cell):
            return pd.NA
        else:
            try:
                return str(int(dataframe_cell))
            except:
                return pd.NA

In [13]:
def drop_columns(df, column_list):
    for col in df.columns:
        if col in column_list:
            df.drop(columns=col, inplace=True)

In [14]:
excel_file_path_sfdc_version = r'C:\Users\phsheari\Documents\SFDC Data\PIPELINE UPDATE\SFDC_Trackerfile.pkl'
compass_file = pd.read_pickle(excel_file_path_sfdc_version)

In [15]:
compass_file['IndexID'] = pd.NA
compass_file['Sheet Name'] = pd.NA

In [16]:
compass_file = compass_file[['Request ID', 'Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'Sales Level 4','GUID', 'Customer Name', 'Deal ID', 'Campaign', 'Compass Advisor','Prehash_Entity',
       'Readout_Date', 'Readout Delivery Status', 'Items Recommended','Campaign Type', 'CONCIERGE_ADVISOR_ASSIGNED','CONCIERGE_INTERNAL_READOUT_DT', ]].drop_duplicates(keep='first')

In [17]:
compass_file['Deal ID'] = compass_file.apply(lambda x: convert_nan_to_int(x['Deal ID']), axis=1)

In [18]:
compass_file['GUID'] = compass_file.apply(lambda x: convert_nan_to_int(x['GUID']), axis=1)

In [19]:
compass_file.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42568 entries, 0 to 42568
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Request ID                     42568 non-null  object        
 1   Sales Level 1                  42567 non-null  object        
 2   Sales Level 2                  42567 non-null  object        
 3   Sales Level 3                  40886 non-null  object        
 4   Sales Level 4                  39414 non-null  object        
 5   GUID                           39481 non-null  object        
 6   Customer Name                  42568 non-null  object        
 7   Deal ID                        20179 non-null  object        
 8   Campaign                       42537 non-null  object        
 9   Compass Advisor                21338 non-null  object        
 10  Prehash_Entity                 42568 non-null  object        
 11  Readout_Date   

In [20]:
today = dt.datetime.today() #.isoformat()[:10]

In [21]:
ten_months_ago = today + dt.timedelta(weeks = -43, days = -3)

In [22]:
compass_file = compass_file.loc[~compass_file['Readout Delivery Status'].isin(['Declined','Postponed']) | (compass_file['CONCIERGE_INTERNAL_READOUT_DT'].notnull())]

In [23]:
ro = cr.get_readout_dataframe()

In [24]:
ro.drop_duplicates(keep="first", inplace=True)

In [25]:
ro = ro.loc[ro['Readout_Date'] > ten_months_ago]

In [26]:
ro.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13689 entries, 251 to 68711
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Request ID    13689 non-null  object        
 1   GUID          13689 non-null  int32         
 2   Readout_Date  13689 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int32(1), object(1)
memory usage: 374.3+ KB


In [27]:
ro = pd.merge(ro, compass_file, how='left', left_on='Request ID', right_on='Request ID')

In [28]:
ro

,Request ID,GUID_x,Readout_Date_x,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,GUID_y,Customer Name,Deal ID,Campaign,Compass Advisor,Prehash_Entity,Readout_Date_y,Readout Delivery Status,Items Recommended,Campaign Type,CONCIERGE_ADVISOR_ASSIGNED,CONCIERGE_INTERNAL_READOUT_DT
0,118238,11438,2022-04-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT
1,118239,11438,2022-04-08,AMERICAS,US PS MARKET SEGMENT,SLED-WEST AREA,SLED-CENTRAL OPERATION,11438,TEXAS BUILDING AND PROCUREMENT COMMISSION,44358639,REACTIVE,Mark Vodka,TE11438,2022-04-08,NaN,NaN,FIELD,NaN,NaT
2,117567,444444,2022-04-12,AMERICAS,AMERICAS_SP,ASP US SEGMENTS,ASP MID MARKET NAMED,444444,BLUE RIDGE COMMUNICATIONS INC,<NA>,COVER THE UNCOVERED - COVERAGE AT RENEWAL,Term,BL444444,2022-04-12,NaN,NaN,COVERAGE AT RENEWAL,NaN,NaT
3,117069,16288,2022-04-19,AMERICAS,US PS MARKET SEGMENT,PS FEDERAL AREA,FED-NATIONAL SECURITY OPERATION,16288,MITRE,<NA>,REACTIVE,Emma Baird,MI16288,2022-04-19,NaN,NaN,FIELD,ebaird@cisco.com,2022-04-18
4,116418,4471862,2022-04-08,AMERICAS,US COMMERCIAL,COMMERCIAL CENTRAL AREA,CENTRAL EAST COMMERCIAL OPERATION,4471862,HENDRICKS,55099391,REACTIVE,Term,HE4471862,2022-04-08,Emailed Recommendations,"BCS,BRW,EC-KT,IRR,SNTC,SSPT,ST",FIELD,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13684,107566,10368781,2021-12-14,AMERICAS,US COMMERCIAL,COMMERCIAL CENTRAL AREA,HEARTLAND-GULF COMMERCIAL OPERATION,10368781,SYNCRO CORPORATION,<NA>,REACTIVE,Sangeetha Sampath,SY10368781,2021-12-14,Complete,"BCS,BRW,DNA,EC-HTOM,IRR,SNTC,SSPT",FIELD,NaN,NaT
13685,107567,320903967,2021-12-14,AMERICAS,US COMMERCIAL,COMMERCIAL CENTRAL AREA,HEARTLAND-GULF COMMERCIAL OPERATION,320903967,ENCOMPASS HEALTH CORPORATION,<NA>,REACTIVE,Sangeetha Sampath,EN320903967,2021-12-14,Complete,"BCS,BRW,IRR,EC-HTOM,SNTC,SSPT",FIELD,NaN,NaT
13686,107568,3773236,2021-12-14,AMERICAS,US COMMERCIAL,COMMERCIAL CENTRAL AREA,HEARTLAND-GULF COMMERCIAL OPERATION,3773236,SLIDELL MEMORIAL HOSPITAL,<NA>,REACTIVE,Sangeetha Sampath,SL3773236,2021-12-14,Complete,"BCS,EC-HTOM,IRR,SNTC,SSPT,ST",FIELD,NaN,NaT
13687,107569,2134315,2021-12-14,AMERICAS,US COMMERCIAL,COMMERCIAL CENTRAL AREA,HEARTLAND-GULF COMMERCIAL OPERATION,2134315,MILLIPORE SIGMA,<NA>,REACTIVE,Sangeetha Sampath,MI2134315,2021-12-14,Complete,"BCS,BRW,DNA,EC-HTOM,IRR,SNTC,SSPT,ST",FIELD,NaN,NaT


In [29]:
compass_drop_columns = ['Readout_Date_x','GUID_y']

In [30]:
drop_columns(ro, compass_drop_columns)

In [31]:
ro.rename(columns={'GUID_x':'GUID', 'Customer Name':'Compass GUID Name','Deal ID':'Compass Deal ID', 'Readout_Date_y':'Readout_Date'}, inplace=True)

In [32]:
# ro.rename(columns={'Deal ID':'Compass Deal ID', 'Readout_Date_y':'Readout_Date'}, inplace=True)

In [33]:
ro.drop(ro.loc[ro['Readout_Date'].isnull()].index, inplace=True)

In [34]:
ro.loc[ro['Sales Level 1'].str.contains('apjc', na=False, case=False),'Sales Level 1'] = 'APJC'

In [35]:
ro.loc[ro['Sales Level 1'].str.contains('apjc', na=False, case=False)]

,Request ID,GUID,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,Compass GUID Name,Compass Deal ID,Campaign,Compass Advisor,Prehash_Entity,Readout_Date,Readout Delivery Status,Items Recommended,Campaign Type,CONCIERGE_ADVISOR_ASSIGNED,CONCIERGE_INTERNAL_READOUT_DT
5,116075,6832603,APJC,ASEAN_AREA,ASEAN_SMALL,ASEAN_SMALL_SGP,DBS CHINA COMPANY LTD,<NA>,REACTIVE,Houman Asefi,DB6832603,2022-04-07,Complete,"BCS,BRW,DNA,EC-AM,EC-HTOM,EC-KT,IRR,PR,SNTC,SS...",FIELD,NaN,NaT
29,114711,6203942,APJC,JAPAN__,JP_ENTERPRISE_CGEM,GLL_MUFG,MUFG US,<NA>,TELEMETRY,Houman Asefi,MU6203942,2022-04-04,Emailed Recommendations,"BCS,BRW,DNA,EC-AM,EC-HTOM,EC-KT,IRR,PR,SNTC,SS...",TELEMETRY,NaN,NaT
160,114449,37472184,APJC,ASEAN_AREA,ASEAN_SG,ASEAN_PS_SGP,COMFORT DELGRO CORP,<NA>,TELEMETRY,Houman Asefi,CO37472184,2022-03-24,Complete,"BCS,BRW,DNA,EC-AM,EC-HTOM,EC-KT,IRR,PR,SNTC,SS...",TELEMETRY,NaN,NaT
473,115447,5485873,APJC,ASEAN_AREA,ASEAN_SG,ASEAN_ENT_SGP,SINGAPORE EXCHANGE LIMITED,44395711,REACTIVE,Houman Asefi,SI5485873,2022-03-22,Complete,"BCS,BRW,DNA,EC-HTOM,EC-KT,IRR,SNTC,SSPT,ST",FIELD,NaN,NaT
872,115662,4104479,APJC,ANZ AREA,ANZ_SMALL,AU_SMALL,NSW HEALTH,<NA>,REACTIVE,Jayjay Towler,NS4104479,2022-04-04,Emailed Recommendations,"BCS,EC-HTOM,EC-AM,EC-KT,IRR,SNTC,SSPT,ST",FIELD,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11687,106584,111454259,APJC,INDIA_AREA,IND_ENTERPRISE_MAJOR,IND_ENT_BFSI_PVT,YES BANK,55164328,REACTIVE,Houman Asefi,YE111454259,2021-12-08,Complete,"BCS,BRW,DNA,EC-AM,IRR,PR,SNTC,SSPT,ST",FIELD,hasefi@cisco.com,2022-02-03
11688,106627,4544445,APJC,ASEAN_AREA,ASEAN_ID,ASEAN_ENT_IDN,BANK OF THE PHILIPPINE ISLANDS,55171848,REACTIVE,Houman Asefi,BA4544445,2021-12-07,Complete,"BCS,BRW,DNA,EC-HTOM,EC-KT,IRR,SNTC,SSPT,ST",FIELD,NaN,NaT
11696,106267,7322062,APJC,ANZ AREA,AU_PUBLIC SECTOR,AU_PS_VIC_REG,FIRE RESCUE VICTORIA,55476773,REACTIVE,Houman Asefi,FI7322062,2021-12-02,Declined,NaN,FIELD,hasefi@cisco.com,2021-12-02
11697,106632,39975112,APJC,ASEAN_AREA,ASEAN_PH,ASEAN_ENT_PHL,LAND BANK OF THE PHILIPPINES,43496487,REACTIVE,Houman Asefi,LA39975112,2021-12-07,Complete,"BCS,BRW,DNA,EC-AM,IRR,PR,SNTC,SSPT,ST",FIELD,Hasefi@cisco.com,2021-12-07


In [36]:
# sfdc_full_export_path = r'C:\Users\phsheari\Documents\SFDC Data\FULL EXPORT'
# services_source = pd.read_pickle(sfdc_full_export_path + r'\COMBINED.pkl')

In [37]:
# services_source.drop_duplicates(keep='first', inplace=True)

In [38]:
# services_source

In [39]:
today = dt.datetime.today() #.isoformat()[:10]

In [40]:
weekday_dict = {0:1,       # 0 = Monday, 1 = Tuesday, 2 = Wednesday, 3 = Thursday, 4 = Friday
                1:2,
                2:3,
                3:4,
                4:5}

In [41]:
todays_weekday = today.weekday()

In [42]:
todays_weekday

0

In [43]:
if weekC == None:
    weeks_ago = today - dt.timedelta((weeks_length * 7)+(weekday_dict[todays_weekday]))      #Looking two weeks back + 2 days. This is normall run on a Monday, so we want that prior Saturday, 2 weeks ago
    weeks_ago = weeks_ago.isoformat()[:10]
else:
    weeks_ago = today - dt.timedelta((weeks_length * 7)+(weekday_dict[todays_weekday]))      #Looking three weeks back + 2 days. This is normall run on a Monday, so we want that prior Saturday, 3 weeks ago
    weeks_ago = weeks_ago.isoformat()[:10]

In [44]:
print(weeks_ago)

2022-03-27


In [45]:
cal = cfc.get_calendar_dataframe()

In [46]:
today.isoformat()[:10]

'2022-04-11'

In [47]:
cal.loc[cal['date']==today.isoformat()[:10], 'finbi_fiscal_week_id'].values[0]

2022093

In [48]:
start_fiscal_week = cal.loc[cal['date']==weeks_ago, 'finbi_fiscal_week_id'].values[0]
end_fiscal_week = cal.loc[cal['date']==today.isoformat()[:10], 'finbi_fiscal_week_id'].values[0]

In [49]:
print(start_fiscal_week, end_fiscal_week)

2022091 2022093


In [50]:
CONVERTERS = {'End Customer Global Ultimate ID': convert_nan_to_int, 'CAV¬†End Customer ID':convert_nan_to_int , 'Deal ID': convert_nan_to_int}

In [51]:
renamefields = {'End Customer Global Ultimate Name':'GUID Name','ERP Deal ID':'Deal ID', 'New/Renewal':'Service Source',
       'End Customer Global Ultimate ID':'GUID', 'CAV End Customer Name':'CAV Name',
       'CAVÂ End Customer ID':'CAV ID', 'CAV End Customer ID':'CAV ID', 'CA Service Bookings Net':'TCV', 'Annual Bookings Net':'ACV'}

In [52]:
fin_path = r'C:\Users\phsheari\Documents\FIN BI Data\2_APJC'
entries = os.listdir(fin_path)

In [53]:
BCSlist = []

for f_name in entries:
    if f_name.startswith('APJC_AS') and f_name.endswith('.xlsx'):
        tempdf = pd.read_excel(fin_path + "\\" + f_name, skiprows=3, converters = CONVERTERS)
        BCSlist.append(tempdf)

C:\Users\phsheari\.conda\envs\compass_env\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [54]:
bcs = pd.concat(BCSlist)

In [55]:
bcs['AS Offer Type']='BCS'

In [56]:
bcs.columns

Index(['Unnamed: 0', 'Fiscal Year', 'Fiscal Quarter ID', 'Fiscal Period ID',
       'Unnamed: 4', 'Fiscal Week ID', 'Finance BU/Service Category',
       'Finance Product Family/Allctd Svcs', 'Sales Level 1', 'Unnamed: 9',
       'Sales Level 2', 'Sales Level 3', 'Sales Level 4',
       'End Customer Global Ultimate Name', 'End Customer Global Ultimate ID',
       'CAV End Customer Name', 'CAV End Customer ID', 'Deal ID',
       'New/Renewal', 'AS Offer Type', 'Partner Name',
       'Finance Sub Group/Contract Type', 'CX Product Portfolio',
       'CX Product Category', 'CX Product', 'Product ID',
       'CA Service Bookings Net', 'Annual Bookings Net'],
      dtype='object')

In [57]:
bcs.head(3)

,Unnamed: 0,Fiscal Year,Fiscal Quarter ID,Fiscal Period ID,Unnamed: 4,Fiscal Week ID,Finance BU/Service Category,Finance Product Family/Allctd Svcs,Sales Level 1,Unnamed: 9,...,New/Renewal,AS Offer Type,Partner Name,Finance Sub Group/Contract Type,CX Product Portfolio,CX Product Category,CX Product,Product ID,CA Service Bookings Net,Annual Bookings Net
0,NaN,2022,2022Q3,202207,NaN,2022071,ADVANCED SERVICES,AS SUBSCRIPTION,APJC,NaN,...,NEW,BCS,UNKNOWN,RBAS,Business Critical Services,BCS 2.x,BCS 2.x,CON-RBAS-1,"-3,605.08","-3,605.08"
1,NaN,2022,2022Q3,202207,NaN,2022071,ADVANCED SERVICES,AS SUBSCRIPTION,APJC,NaN,...,NEW,BCS,UNKNOWN,RBAS,Business Critical Services,BCS 2.x,BCS 2.x,CON-RBAS-1,"3,840.48","3,840.48"
2,NaN,2022,2022Q3,202207,NaN,2022071,ADVANCED SERVICES,AS SUBSCRIPTION,APJC,NaN,...,NEW,BCS,NTT LTD COMMERCE CENTRE,CXO,Business Critical Services,BCS 3.x,BCS 3.x Other,CON-CXO-OA-AS,0.00,0.00


In [58]:
bcs.rename(columns={'CAV End Customer ID': 'CAV ID'}, inplace=True)

In [59]:
bcs['CX Product Category'].unique()

array(['BCS 2.x', 'BCS 3.x'], dtype=object)

In [60]:
HTEClist = []

for f_name in entries:
    if f_name.startswith('APJC_TS') and f_name.endswith('.xlsx'):
        tempdf = pd.read_excel(fin_path + "\\" + f_name, skiprows=3, converters = CONVERTERS)
        HTEClist.append(tempdf)

C:\Users\phsheari\.conda\envs\compass_env\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [61]:
htec = pd.concat(HTEClist)
# htec['Finance Program Brand'] = 'HTEC'
htec.rename(columns={'Finance Program Brand':'AS Offer Type'}, inplace=True)

In [62]:
htec['CX Product Category'].unique()

array(['Solution Support', 'Expert Care'], dtype=object)

In [63]:
def get_offer_type(cx_product_category):
    if pd.isnull(cx_product_category):
        return pd.NA
    elif cx_product_category.lower() == 'expert care':
        return 'HTEC'
    elif cx_product_category.lower() == 'solution support':
        return 'Solution Support'
    else:
        return pd.NA

In [64]:
htec.rename(columns={'CAV End Customer ID': 'CAV ID'}, inplace=True)

In [65]:
htec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104916 entries, 0 to 104915
Data columns (total 23 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Unnamed: 0                         0 non-null       float64
 1   Fiscal Year                        104916 non-null  int64  
 2   Fiscal Quarter ID                  104916 non-null  object 
 3   Fiscal Period ID                   104916 non-null  int64  
 4   Fiscal Week ID                     104916 non-null  int64  
 5   Finance BU/Service Category        104916 non-null  object 
 6   Sales Level 1                      104916 non-null  object 
 7   Sales Level 2                      104916 non-null  object 
 8   Sales Level 3                      104916 non-null  object 
 9   Sales Level 4                      104916 non-null  object 
 10  End Customer Global Ultimate Name  104916 non-null  object 
 11  End Customer Global Ultimate ID    1045

In [66]:
htec['AS Offer Type'] = htec['CX Product Category'].apply(get_offer_type)

In [67]:
htec.shape

(104916, 24)

In [68]:
Successlist = []

for f_name in entries:
    if f_name.startswith('APJC_Success') and f_name.endswith('.xlsx'):
        tempdf = pd.read_excel(fin_path + "\\" + f_name, skiprows=3, converters = CONVERTERS)
        Successlist.append(tempdf)

In [69]:
if Successlist:
    success = pd.concat(Successlist)
else:
    success = None

In [70]:
success.rename(columns={'CAV End Customer ID': 'CAV ID'}, inplace=True)

In [71]:
success.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1578 entries, 0 to 1577
Data columns (total 23 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Unnamed: 0                         0 non-null      float64
 1   Fiscal Year                        1578 non-null   int64  
 2   Fiscal Quarter ID                  1578 non-null   object 
 3   Fiscal Period ID                   1578 non-null   int64  
 4   Fiscal Week ID                     1578 non-null   int64  
 5   Finance BU/Service Category        1578 non-null   object 
 6   Sales Level 1                      1578 non-null   object 
 7   Sales Level 2                      1578 non-null   object 
 8   Sales Level 3                      1578 non-null   object 
 9   Sales Level 4                      1578 non-null   object 
 10  End Customer Global Ultimate Name  1578 non-null   object 
 11  End Customer Global Ultimate ID    1577 non-null   objec

In [72]:
success.shape

(1578, 23)

In [73]:
# if success is not None:
#     success = success.loc[success['Fiscal Week ID'] >= test_fiscal_week]

In [74]:
SNTC_list = []

for f_name in entries:
    if f_name.startswith('APJC_SNTC') and f_name.endswith('.xlsx'): 
        tempdf = pd.read_excel(fin_path + "\\" + f_name, skiprows=3, converters = CONVERTERS)
        SNTC_list.append(tempdf)

In [75]:
sntc = pd.concat(SNTC_list)
sntc.rename(columns={'Finance Program Brand':'AS Offer Type'}, inplace=True)

In [76]:
sntc.rename(columns=renamefields, inplace=True)

In [77]:
sntc.shape

(192001, 23)

In [78]:
dropcolumns = ['Unnamed: 0','Finance Product Sub Group Description','Finance Product Family/Allctd Svcs', 'Sales Level 5',
       'Sales Level 6', 'SSS Flag', 'SPM Coverage','DE/PSSM Flag','PDE/PSSM Flag','Contract Number','Finance Sub Group/Contract Type','CX Product Portfolio','CX Product']

In [79]:
for col in success.columns:
    if col in renamefields.keys():
        success.rename(columns = {col:renamefields[col]}, inplace=True)

In [80]:
drop_columns(success, dropcolumns)

In [81]:
drop_columns(bcs, dropcolumns)

In [82]:
drop_columns(htec, dropcolumns)

In [83]:
for col in bcs.columns:
    if col in renamefields.keys():
        bcs.rename(columns={col:renamefields[col]}, inplace=True)

In [84]:
for col in htec.columns:
    if col in renamefields.keys():
        htec.rename(columns={col:renamefields[col]}, inplace=True)

In [85]:
for col in sntc.columns:
    if col in renamefields.keys():
        sntc.rename(columns={col:renamefields[col]}, inplace=True)

In [86]:
success.columns

Index(['Fiscal Year', 'Fiscal Quarter ID', 'Fiscal Period ID',
       'Fiscal Week ID', 'Finance BU/Service Category', 'Sales Level 1',
       'Sales Level 2', 'Sales Level 3', 'Sales Level 4', 'GUID Name', 'GUID',
       'CAV Name', 'CAV End Customer ID', 'Deal ID', 'Service Source',
       'Partner Name', 'CX Product Category', 'Product ID', 'TCV', 'ACV'],
      dtype='object')

In [87]:
success['CX Product Category'].unique()

array(['Level 2', 'Level 1'], dtype=object)

In [88]:
success['AS Offer Type']='SuccessTracks'
success['CX Product Category']='SuccessTracks'

In [89]:
sntc['AS Offer Type']='SNTC'

In [90]:
bcs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 682 entries, 0 to 681
Data columns (total 23 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Fiscal Year                  682 non-null    int64  
 1   Fiscal Quarter ID            682 non-null    object 
 2   Fiscal Period ID             682 non-null    int64  
 3   Unnamed: 4                   0 non-null      float64
 4   Fiscal Week ID               682 non-null    int64  
 5   Finance BU/Service Category  682 non-null    object 
 6   Sales Level 1                682 non-null    object 
 7   Unnamed: 9                   0 non-null      float64
 8   Sales Level 2                682 non-null    object 
 9   Sales Level 3                682 non-null    object 
 10  Sales Level 4                682 non-null    object 
 11  GUID Name                    682 non-null    object 
 12  GUID                         682 non-null    int64  
 13  CAV Name            

In [91]:
htec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104916 entries, 0 to 104915
Data columns (total 21 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Fiscal Year                  104916 non-null  int64  
 1   Fiscal Quarter ID            104916 non-null  object 
 2   Fiscal Period ID             104916 non-null  int64  
 3   Fiscal Week ID               104916 non-null  int64  
 4   Finance BU/Service Category  104916 non-null  object 
 5   Sales Level 1                104916 non-null  object 
 6   Sales Level 2                104916 non-null  object 
 7   Sales Level 3                104916 non-null  object 
 8   Sales Level 4                104916 non-null  object 
 9   GUID Name                    104916 non-null  object 
 10  GUID                         104532 non-null  object 
 11  CAV Name                     104916 non-null  object 
 12  CAV End Customer ID          104916 non-null  int64  
 13 

In [92]:
success.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1578 entries, 0 to 1577
Data columns (total 21 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Fiscal Year                  1578 non-null   int64  
 1   Fiscal Quarter ID            1578 non-null   object 
 2   Fiscal Period ID             1578 non-null   int64  
 3   Fiscal Week ID               1578 non-null   int64  
 4   Finance BU/Service Category  1578 non-null   object 
 5   Sales Level 1                1578 non-null   object 
 6   Sales Level 2                1578 non-null   object 
 7   Sales Level 3                1578 non-null   object 
 8   Sales Level 4                1578 non-null   object 
 9   GUID Name                    1578 non-null   object 
 10  GUID                         1577 non-null   object 
 11  CAV Name                     1578 non-null   object 
 12  CAV End Customer ID          1578 non-null   int64  
 13  Deal ID           

In [93]:
success = success.reindex(columns=bcs.columns)

In [94]:
success.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1578 entries, 0 to 1577
Data columns (total 23 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Fiscal Year                  1578 non-null   int64  
 1   Fiscal Quarter ID            1578 non-null   object 
 2   Fiscal Period ID             1578 non-null   int64  
 3   Unnamed: 4                   0 non-null      float64
 4   Fiscal Week ID               1578 non-null   int64  
 5   Finance BU/Service Category  1578 non-null   object 
 6   Sales Level 1                1578 non-null   object 
 7   Unnamed: 9                   0 non-null      float64
 8   Sales Level 2                1578 non-null   object 
 9   Sales Level 3                1578 non-null   object 
 10  Sales Level 4                1578 non-null   object 
 11  GUID Name                    1578 non-null   object 
 12  GUID                         1577 non-null   object 
 13  CAV Name          

In [95]:
sntc = sntc.reindex(columns=bcs.columns)

In [96]:
sntc.shape

(192001, 23)

In [97]:
htec = htec.reindex(columns=bcs.columns)

In [98]:
df = pd.concat([bcs,htec,success, sntc])

In [99]:
df = df.loc[(df['Fiscal Week ID']>= start_fiscal_week) & (df['Fiscal Week ID'] < end_fiscal_week)]

In [100]:
df['Fiscal Week ID'].unique()

array([2022091, 2022092], dtype=int64)

In [101]:
df['ReportWeek'] = df['Fiscal Week ID'].apply(get_reporting_week)

In [102]:
new_column_list = []
for col in df.columns:
    if '\u00C2' in col:
        col = col.replace('\u00C2','').replace('\xa0',' ').replace('\\xa0',' ')
    new_column_list.append(col)

In [103]:
# new_column_list

In [104]:
for i,element in enumerate(new_column_list):
    if element=='CAV End Customer ID':
        new_column_list[i] = 'CAV ID'
        
# new_column_list    

In [105]:
df.columns = new_column_list

In [106]:
df.rename(columns={'CAV\xa0End Customer ID':'CAV ID'}, inplace=True)

In [107]:
df.columns

Index(['Fiscal Year', 'Fiscal Quarter ID', 'Fiscal Period ID', 'Unnamed: 4',
       'Fiscal Week ID', 'Finance BU/Service Category', 'Sales Level 1',
       'Unnamed: 9', 'Sales Level 2', 'Sales Level 3', 'Sales Level 4',
       'GUID Name', 'GUID', 'CAV Name', 'CAV ID', 'Deal ID', 'Service Source',
       'AS Offer Type', 'Partner Name', 'CX Product Category', 'Product ID',
       'TCV', 'ACV', 'ReportWeek'],
      dtype='object')

In [108]:
df['Deal ID'] = df['Deal ID'].apply(convert_nan_to_int)

In [109]:
df['CAV ID'] = df['CAV ID'].apply(convert_nan_to_int)

In [110]:
df['GUID'] = df['GUID'].apply(convert_nan_to_int)

In [111]:
def squeeze_customer_name(customer_name):
    if customer_name.startswith('THE '):
        customer_name = customer_name[4:]
    customer_name = customer_name.replace(',',"").replace('[ACTIVE]','').replace('-','').replace('_','').replace('.','').replace(' ','').strip()
    return customer_name

In [112]:
### Start with GUID. If GUID is None, use CAV ID, if CAV ID, is None, use CustomerName, if GUID is bad or a list with unacceptable alpha lexicon values, use CustomerName
def make_fake_guid(guid, cavid, customername):
    if pd.isnull(guid):
        if pd.isnull(cavid) or cavid == -999:
            return str(customername[:10])
        else:
            if not isinstance(cavid, int):
                cavid = cavid.split(',')
                return str(cavid[0])
    elif not isinstance(guid,int):
        guid = guid.split(',')
        return str(guid[0])
    else:
        return str(guid)

In [113]:
df['Squeezed'] = df['CAV Name'].apply(lambda x: squeeze_customer_name(x))

In [114]:
df['Prepend'] = df['Squeezed'].apply(lambda x: x[:2].upper())

In [115]:
df['FakeID'] = df.apply(lambda x: make_fake_guid(x['GUID'], x['CAV ID'], x['Squeezed']), axis=1)

In [116]:
df.loc[df['FakeID'].isnull()][['GUID','CAV ID','CAV Name', 'Squeezed','Prepend','FakeID']]

,GUID,CAV ID,CAV Name,Squeezed,Prepend,FakeID


In [117]:
#Prehash_Entity
df['Prehash_Entity'] = df.apply(lambda x: x['Prepend'] + x['FakeID'], axis=1)

In [118]:
df.columns

Index(['Fiscal Year', 'Fiscal Quarter ID', 'Fiscal Period ID', 'Unnamed: 4',
       'Fiscal Week ID', 'Finance BU/Service Category', 'Sales Level 1',
       'Unnamed: 9', 'Sales Level 2', 'Sales Level 3', 'Sales Level 4',
       'GUID Name', 'GUID', 'CAV Name', 'CAV ID', 'Deal ID', 'Service Source',
       'AS Offer Type', 'Partner Name', 'CX Product Category', 'Product ID',
       'TCV', 'ACV', 'ReportWeek', 'Squeezed', 'Prepend', 'FakeID',
       'Prehash_Entity'],
      dtype='object')

In [119]:
df['Deal ID'] = df['Deal ID'].fillna(value=-9999, axis = 0)

In [120]:
df = df[['Fiscal Year', 'Fiscal Quarter ID', 'Fiscal Period ID',
       'Fiscal Week ID', 'ReportWeek', 'Finance BU/Service Category', 'Sales Level 1',
       'Sales Level 2', 'Sales Level 3', 'Sales Level 4', 'CAV ID', 'CAV Name', 'GUID', 'GUID Name', 'Partner Name',
       'Deal ID', 'AS Offer Type', 'CX Product Category', 'Product ID', 'TCV', 'ACV', 'Service Source', 'Squeezed','Prepend','FakeID', 'Prehash_Entity']]

In [121]:
df.rename(columns = {'Deal ID':'FINBI Deal ID','Partner Name':'FINBI Partner'}, inplace=True)

In [122]:
table = pd.pivot_table(df, values = ['ACV','TCV'], index=['Sales Level 1', 'Sales Level 2',
       'Sales Level 3', 'Sales Level 4', 'CAV ID', 'CAV Name', 'GUID', 'GUID Name',  
       'FINBI Deal ID', 'FINBI Partner','AS Offer Type', 'CX Product Category', 'Fiscal Quarter ID','Fiscal Week ID', 'ReportWeek','Service Source','Squeezed','Prepend','FakeID', 'Prehash_Entity'], aggfunc='sum')

In [123]:
# table.reset_index(inplace=True)

In [124]:
# df.loc[df['GUID Name']=='VANCOUVER ISLAND HEALTH AUTHORITY']

In [125]:
table

ACV  \
Sales Level 1 Sales Level 2 Sales Level 3 Sales Level 4 CAV ID   CAV Name             GUID       GUID Name               FINBI Deal ID FINBI Partner          AS Offer Type    CX Product Category Fiscal Quarter ID Fiscal Week ID ReportWeek Service Source Squeezed            Prepend FakeID    Prehash_Entity             
APJC          ANZ AREA      ANZ_ MISC     ANZ_MISC OPS  -999     UNKNOWN              -999       UNKNOWN                 -9999         BES IT Systems         Solution Support Solution Support    2022Q3            2022091        9          NEW            UNKNOWN             UN      -999      UN-999              0.00   
                                                                                                                                       UNKNOWN                BCS              BCS 2.x             2022Q3            2022091        9          NEW            UNKNOWN             UN      -999      UN-999         -9,709.97   
                                                                                       146798    KAO CORPORATION         -9999         LOGICALIS              SNTC             SNTC                2022Q3            2022091        9          RENEW          UNKNOWN             UN      146798    UN146798            0.00   
                                                                                                                                       UNKNOWN                SNTC             SNTC                2022Q3            2022091        9          RENEW          UNKNOWN             UN      146798    UN146798            0.00   
                                                                                       4104479   COMMONWEALTH GOVERNMENT -9999         DATA#3 LIMITED         SNTC             SNTC                2022Q3            2022091        9          RENEW          UNKNOWN             UN      4104479   UN4104479           0.00   
...                                                                                                                                                                                                                                                                                                                      ...   
              ROK_AREA      KOR_SMALL     KR_SMALL_SBU   7960598 HEUNG K HOSPITAL      380544374 HEUNG K HOSPITAL        -9999         Newone systems         SNTC             SNTC                2022Q3            2022091        9          NEW            HEUNGKHOSPITAL      HE      380544374 HE380544374       119.59   
                                                         7961565 OBS BANK              380734935 OBS BANK                -9999         e-Tech System Co., Ltd SNTC             SNTC                2022Q3            2022091        9          NEW            OBSBANK             OB      380734935 OB380734935       620.29   
                                                         7964808 HONGMOON CORPORATION  380776954 HONGMOON CORPORATION    -9999         Ace Net                SNTC             SNTC                2022Q3            2022091        9          NEW            HONGMOONCORPORATION HO      380776954 HO380776954        75.85   
                                                         7965890 SM RADIOLOGY CLINIC   378406103 SM RADIOLOGY CLINIC      53634786     U-tas, .LTD            SNTC             SNTC                2022Q3            2022092        10         NEW            SMRADIOLOGYCLINIC   SM      378406103 SM378406103         0.00   
                                                         7967414 LTM ACENETPIA         380971138 LTM ACENETPIA           -9999         Ace Net                SNTC             SNTC                2022Q3            2022092        10         NEW            LTMACENETPIA        LT      380971138 LT380971138        49.92   

                                                                                                                                                        

In [126]:
# table.loc[table['Bookings']>=1]

In [127]:
tableP = table.loc[table['TCV']>0]

In [128]:
tableP['Ind'] = 'P'

<ipython-input-128-09d0920af551>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tableP['Ind'] = 'P'


In [129]:
tableN = table.loc[table['TCV']<=0]

In [130]:
tableN['Ind'] = 'N'

<ipython-input-130-7a79b8daff69>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tableN['Ind'] = 'N'


In [131]:
tableN.reset_index(inplace=True) #['Deal ID']== -9999

In [132]:
# tableN

In [133]:
### This is DROP Criteria; i.e. > -10,000 to 0

tableN.drop(tableN.loc[(tableN['TCV'] > -12000) & (tableN['TCV'] <= 0)].index, inplace=True)   #tableN.drop(((tableN['TCV'] > -20000) & (tableN['TCV'] <= 0))].index, inplace=True) 

C:\Users\phsheari\.conda\envs\compass_env\lib\site-packages\pandas\core\frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [134]:
tableN

,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,CAV ID,CAV Name,GUID,GUID Name,FINBI Deal ID,FINBI Partner,...,Fiscal Week ID,ReportWeek,Service Source,Squeezed,Prepend,FakeID,Prehash_Entity,ACV,TCV,Ind
425,APJC,ANZ AREA,ANZ_CONTRA,ANZ_CONTRA_OPS,100665,INSURANCE AUSTRALIA GROUP LTD,5883288,INSURANCE AUSTRALIA GROUP LTD,56027888,UNKNOWN,...,2022092,10,NEW,INSURANCEAUSTRALIAGROUPLTD,IN,5883288,IN5883288,"-4,575.77","-22,878.84",N
428,APJC,ANZ AREA,ANZ_CONTRA,ANZ_CONTRA_OPS,100675,BHP BILLITON LTD,156215,BHP BILLITON LTD,-9999,UNKNOWN,...,2022092,10,RENEW,BHPBILLITONLTD,BH,156215,BH156215,"-25,263.04","-25,781.77",N
491,APJC,ANZ AREA,ANZ_CONTRA,ANZ_CONTRA_OPS,103142,JONES LANG LASALLE INC,21175,JONES LANG LASALLE INC,53987660,UNKNOWN,...,2022092,10,NEW,JONESLANGLASALLEINC,JO,21175,JO21175,"-3,213.12","-12,870.21",N
539,APJC,ANZ AREA,ANZ_CONTRA,ANZ_CONTRA_OPS,107865,SWINBURNE UNIVERSITY OF TECHNOLOGY,6163312,SWINBURNE UNIVERSITY OF TECHNOLOGY,-9999,UNKNOWN,...,2022091,9,NEW,SWINBURNEUNIVERSITYOFTECHNOLOGY,SW,6163312,SW6163312,"-64,020.78","-64,020.78",N
541,APJC,ANZ AREA,ANZ_CONTRA,ANZ_CONTRA_OPS,107865,SWINBURNE UNIVERSITY OF TECHNOLOGY,6163312,SWINBURNE UNIVERSITY OF TECHNOLOGY,44257385,UNKNOWN,...,2022092,10,RENEW,SWINBURNEUNIVERSITYOFTECHNOLOGY,SW,6163312,SW6163312,"-24,874.06","-24,874.06",N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41185,APJC,ROK_AREA,KOR_ENTERPRISE_MAJOR,KR_ENT_MAJOR,173543,NATIONAL AGRICULTURAL COOPERATIVE FEDERATION,137872861,NATIONAL AGRICULTURAL COOPERATIVE FEDERATION S...,55525464,UNKNOWN,...,2022091,9,NEW,NATIONALAGRICULTURALCOOPERATIVEFEDERATION,NA,137872861,NA137872861,"-75,773.96","-75,773.96",N
41246,APJC,ROK_AREA,KOR_ENTERPRISE_MAJOR,KR_ENT_MAJOR,201467,COUPANG COLTD,210668982,COUPANG COLTD,55749818,AIRQUAY,...,2022092,10,NEW,COUPANGCOLTD,CO,210668982,CO210668982,"-17,707.39","-17,707.39",N
41763,APJC,ROK_AREA,KOR_PUBLIC SECTOR,KR_PS_CENTRAL,1809767,KOREA AIRPORT CORPORATION,4445502,GOVERNMENT OF THE REPUBLIC OF KOREA,20228141,SNET SYSTEMS INC.,...,2022092,10,NEW,KOREAAIRPORTCORPORATION,KO,4445502,KO4445502,"-10,608.32","-31,824.95",N
42013,APJC,ROK_AREA,KOR_SMALL,KR_SMALL_SBU,640947,THE DAEGU BANK LTD,4543414,THE DAEGU BANK LTD,44228135,"Sangsangin Co., Ltd.",...,2022091,9,RENEW,DAEGUBANKLTD,DA,4543414,DA4543414,"-72,132.03","-72,132.04",N


In [135]:
tableP.reset_index(inplace=True)

In [136]:
bcs_condition = ((tableP['TCV']>= 10000) & (tableP['AS Offer Type']=='BCS'))
htec_condition = ((tableP['TCV']>= 10000) & (tableP['AS Offer Type']=='HTEC'))
ssspt_condition = ((tableP['TCV']>= 10000) & (tableP['AS Offer Type']=='Solution Support'))
st_condition = ((tableP['TCV']>= 500) & (tableP['AS Offer Type']=='SuccessTracks'))

In [137]:
tableP = tableP.loc[bcs_condition | htec_condition | ssspt_condition | st_condition]

In [138]:
table = pd.concat([tableN,tableP])

In [139]:
# table = table.loc[~(table['Deal ID']==-9999)]

In [140]:
# table

In [141]:
table.sort_values(by=['Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'Sales Level 4', 'CAV ID', 'CAV Name',
       'GUID', 'GUID Name','FINBI Deal ID','Service Source','Ind'], inplace=True)

In [142]:
table

,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,CAV ID,CAV Name,GUID,GUID Name,FINBI Deal ID,FINBI Partner,...,Fiscal Week ID,ReportWeek,Service Source,Squeezed,Prepend,FakeID,Prehash_Entity,ACV,TCV,Ind
425,APJC,ANZ AREA,ANZ_CONTRA,ANZ_CONTRA_OPS,100665,INSURANCE AUSTRALIA GROUP LTD,5883288,INSURANCE AUSTRALIA GROUP LTD,56027888,UNKNOWN,...,2022092,10,NEW,INSURANCEAUSTRALIAGROUPLTD,IN,5883288,IN5883288,"-4,575.77","-22,878.84",N
428,APJC,ANZ AREA,ANZ_CONTRA,ANZ_CONTRA_OPS,100675,BHP BILLITON LTD,156215,BHP BILLITON LTD,-9999,UNKNOWN,...,2022092,10,RENEW,BHPBILLITONLTD,BH,156215,BH156215,"-25,263.04","-25,781.77",N
491,APJC,ANZ AREA,ANZ_CONTRA,ANZ_CONTRA_OPS,103142,JONES LANG LASALLE INC,21175,JONES LANG LASALLE INC,53987660,UNKNOWN,...,2022092,10,NEW,JONESLANGLASALLEINC,JO,21175,JO21175,"-3,213.12","-12,870.21",N
539,APJC,ANZ AREA,ANZ_CONTRA,ANZ_CONTRA_OPS,107865,SWINBURNE UNIVERSITY OF TECHNOLOGY,6163312,SWINBURNE UNIVERSITY OF TECHNOLOGY,-9999,UNKNOWN,...,2022091,9,NEW,SWINBURNEUNIVERSITYOFTECHNOLOGY,SW,6163312,SW6163312,"-64,020.78","-64,020.78",N
541,APJC,ANZ AREA,ANZ_CONTRA,ANZ_CONTRA_OPS,107865,SWINBURNE UNIVERSITY OF TECHNOLOGY,6163312,SWINBURNE UNIVERSITY OF TECHNOLOGY,44257385,UNKNOWN,...,2022092,10,RENEW,SWINBURNEUNIVERSITYOFTECHNOLOGY,SW,6163312,SW6163312,"-24,874.06","-24,874.06",N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41763,APJC,ROK_AREA,KOR_PUBLIC SECTOR,KR_PS_CENTRAL,1809767,KOREA AIRPORT CORPORATION,4445502,GOVERNMENT OF THE REPUBLIC OF KOREA,20228141,SNET SYSTEMS INC.,...,2022092,10,NEW,KOREAAIRPORTCORPORATION,KO,4445502,KO4445502,"-10,608.32","-31,824.95",N
42013,APJC,ROK_AREA,KOR_SMALL,KR_SMALL_SBU,640947,THE DAEGU BANK LTD,4543414,THE DAEGU BANK LTD,44228135,"Sangsangin Co., Ltd.",...,2022091,9,RENEW,DAEGUBANKLTD,DA,4543414,DA4543414,"-72,132.03","-72,132.04",N
42073,APJC,ROK_AREA,KOR_SMALL,KR_SMALL_SBU,2550366,TELECHIPS INC.,271853584,TELECHIPS INC.,55270165,UNKNOWN,...,2022092,10,NEW,TELECHIPSINC,TE,271853584,TE271853584,"-30,421.46","-152,107.32",N
9014,APJC,ROK_AREA,KOR_SMALL,KR_SMALL_SBU,2550366,TELECHIPS INC.,271853584,TELECHIPS INC.,55270165,INSUNG INFORMATION CO LTD,...,2022092,10,NEW,TELECHIPSINC,TE,271853584,TE271853584,"26,467.10","132,335.50",P


In [143]:
# ro.rename(columns = {'Customer Name':'GUID Name', 'Deal ID':'Compass Deal ID'}, inplace=True)

In [144]:
# ro['TimeDelta'] = ro['Readout_Date'].apply(lambda x: today.date() - x)

In [145]:
ro

,Request ID,GUID,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,Compass GUID Name,Compass Deal ID,Campaign,Compass Advisor,Prehash_Entity,Readout_Date,Readout Delivery Status,Items Recommended,Campaign Type,CONCIERGE_ADVISOR_ASSIGNED,CONCIERGE_INTERNAL_READOUT_DT
1,118239,11438,AMERICAS,US PS MARKET SEGMENT,SLED-WEST AREA,SLED-CENTRAL OPERATION,TEXAS BUILDING AND PROCUREMENT COMMISSION,44358639,REACTIVE,Mark Vodka,TE11438,2022-04-08,NaN,NaN,FIELD,NaN,NaT
2,117567,444444,AMERICAS,AMERICAS_SP,ASP US SEGMENTS,ASP MID MARKET NAMED,BLUE RIDGE COMMUNICATIONS INC,<NA>,COVER THE UNCOVERED - COVERAGE AT RENEWAL,Term,BL444444,2022-04-12,NaN,NaN,COVERAGE AT RENEWAL,NaN,NaT
3,117069,16288,AMERICAS,US PS MARKET SEGMENT,PS FEDERAL AREA,FED-NATIONAL SECURITY OPERATION,MITRE,<NA>,REACTIVE,Emma Baird,MI16288,2022-04-19,NaN,NaN,FIELD,ebaird@cisco.com,2022-04-18
4,116418,4471862,AMERICAS,US COMMERCIAL,COMMERCIAL CENTRAL AREA,CENTRAL EAST COMMERCIAL OPERATION,HENDRICKS,55099391,REACTIVE,Term,HE4471862,2022-04-08,Emailed Recommendations,"BCS,BRW,EC-KT,IRR,SNTC,SSPT,ST",FIELD,NaN,NaT
5,116075,6832603,APJC,ASEAN_AREA,ASEAN_SMALL,ASEAN_SMALL_SGP,DBS CHINA COMPANY LTD,<NA>,REACTIVE,Houman Asefi,DB6832603,2022-04-07,Complete,"BCS,BRW,DNA,EC-AM,EC-HTOM,EC-KT,IRR,PR,SNTC,SS...",FIELD,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13684,107566,10368781,AMERICAS,US COMMERCIAL,COMMERCIAL CENTRAL AREA,HEARTLAND-GULF COMMERCIAL OPERATION,SYNCRO CORPORATION,<NA>,REACTIVE,Sangeetha Sampath,SY10368781,2021-12-14,Complete,"BCS,BRW,DNA,EC-HTOM,IRR,SNTC,SSPT",FIELD,NaN,NaT
13685,107567,320903967,AMERICAS,US COMMERCIAL,COMMERCIAL CENTRAL AREA,HEARTLAND-GULF COMMERCIAL OPERATION,ENCOMPASS HEALTH CORPORATION,<NA>,REACTIVE,Sangeetha Sampath,EN320903967,2021-12-14,Complete,"BCS,BRW,IRR,EC-HTOM,SNTC,SSPT",FIELD,NaN,NaT
13686,107568,3773236,AMERICAS,US COMMERCIAL,COMMERCIAL CENTRAL AREA,HEARTLAND-GULF COMMERCIAL OPERATION,SLIDELL MEMORIAL HOSPITAL,<NA>,REACTIVE,Sangeetha Sampath,SL3773236,2021-12-14,Complete,"BCS,EC-HTOM,IRR,SNTC,SSPT,ST",FIELD,NaN,NaT
13687,107569,2134315,AMERICAS,US COMMERCIAL,COMMERCIAL CENTRAL AREA,HEARTLAND-GULF COMMERCIAL OPERATION,MILLIPORE SIGMA,<NA>,REACTIVE,Sangeetha Sampath,MI2134315,2021-12-14,Complete,"BCS,BRW,DNA,EC-HTOM,IRR,SNTC,SSPT,ST",FIELD,NaN,NaT


In [146]:
def get_minimum_date_from_weekid(weekid):
    return cal.loc[cal['finbi_fiscal_week_id']==weekid, 'date'].max()

In [147]:
fiscalweeks = list(table['Fiscal Week ID'].unique())

In [148]:
fiscalweeks.sort()

In [149]:
cal = cal.loc[cal['finbi_fiscal_week_id'].isin(fiscalweeks)]

In [150]:
### Add a time delta function here to get back the 30 days PRIOR to this min date for each fiscal week
min_dates_dict = {}
for x in fiscalweeks:
    min_dates_dict[x] = get_minimum_date_from_weekid(x)

min_dates_dict

{2022091: '2022-04-02', 2022092: '2022-04-09'}

In [151]:
must_be_prior_to_dict = {}
for k, v in min_dates_dict.items():
    must_be_prior_to_dict[k] = (pd.to_datetime(v) + dt.timedelta(days = -30))

In [152]:
must_be_prior_to_dict

{2022091: Timestamp('2022-03-03 00:00:00'),
 2022092: Timestamp('2022-03-10 00:00:00')}

In [153]:
ro['Readout_Date'] = pd.to_datetime(ro['Readout_Date'])

In [154]:
ro['CONCIERGE_INTERNAL_READOUT_DT'] = pd.to_datetime(ro['CONCIERGE_INTERNAL_READOUT_DT'])

In [155]:
ro.loc[ro['Readout_Date'].isnull()][['Request ID','Readout_Date']]

,Request ID,Readout_Date


In [156]:
ro['Readout_Date'] = ro['Readout_Date'].astype(str).replace('-','')

In [157]:
ro = ro.loc[(ro['Readout_Date'].notnull()) & (ro['Sales Level 1']=='APJC') & (~(ro['Readout Delivery Status'].isin(['Declined','Postponed'])) & (ro['Readout Delivery Status'].notnull()))]  #

In [158]:
MaxList = pd.DataFrame(ro.groupby('Compass GUID Name')['Readout_Date'].min())   #should be minimum date to get the best possible date differential

In [159]:
MaxList.reset_index(inplace=True)

In [160]:
MaxList

,Compass GUID Name,Readout_Date
0,ACCOLADE WINES AUSTRALIA LTD,2022-02-09
1,ADAMA INDIA PVT. LTD.,2021-12-09
2,ADANI,2021-07-15
3,ADITYA BIRLA CAPITAL LTD,2022-01-19
4,AFLAC INC,2021-12-06
...,...,...
310,YAMAHA MOTOR CO. LTD.,2021-11-22
311,YANTAI WANHUA GROUP,2021-06-21
312,YES BANK,2021-06-21
313,YOKOGAWA ELECTRIC CORPORATION,2021-10-29


In [161]:
output = pd.merge(ro,MaxList, how='inner', left_on = 'Compass GUID Name', right_on='Compass GUID Name').drop(columns = ['Readout_Date_y'])#.rename(columns={'Readout_Date_x':'Readout_Date'}, inplace=True)

In [162]:
output.rename(columns={'Readout_Date_x':'Readout_Date'}, inplace=True)

In [163]:
output

,Request ID,GUID,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,Compass GUID Name,Compass Deal ID,Campaign,Compass Advisor,Prehash_Entity,Readout_Date,Readout Delivery Status,Items Recommended,Campaign Type,CONCIERGE_ADVISOR_ASSIGNED,CONCIERGE_INTERNAL_READOUT_DT
0,116075,6832603,APJC,ASEAN_AREA,ASEAN_SMALL,ASEAN_SMALL_SGP,DBS CHINA COMPANY LTD,<NA>,REACTIVE,Houman Asefi,DB6832603,2022-04-07,Complete,"BCS,BRW,DNA,EC-AM,EC-HTOM,EC-KT,IRR,PR,SNTC,SS...",FIELD,NaN,NaT
1,114711,6203942,APJC,JAPAN__,JP_ENTERPRISE_CGEM,GLL_MUFG,MUFG US,<NA>,TELEMETRY,Houman Asefi,MU6203942,2022-04-04,Emailed Recommendations,"BCS,BRW,DNA,EC-AM,EC-HTOM,EC-KT,IRR,PR,SNTC,SS...",TELEMETRY,NaN,NaT
2,114449,37472184,APJC,ASEAN_AREA,ASEAN_SG,ASEAN_PS_SGP,COMFORT DELGRO CORP,<NA>,TELEMETRY,Houman Asefi,CO37472184,2022-03-24,Complete,"BCS,BRW,DNA,EC-AM,EC-HTOM,EC-KT,IRR,PR,SNTC,SS...",TELEMETRY,NaN,NaT
3,115447,5485873,APJC,ASEAN_AREA,ASEAN_SG,ASEAN_ENT_SGP,SINGAPORE EXCHANGE LIMITED,44395711,REACTIVE,Houman Asefi,SI5485873,2022-03-22,Complete,"BCS,BRW,DNA,EC-HTOM,EC-KT,IRR,SNTC,SSPT,ST",FIELD,NaN,NaT
4,115662,4104479,APJC,ANZ AREA,ANZ_SMALL,AU_SMALL,NSW HEALTH,<NA>,REACTIVE,Jayjay Towler,NS4104479,2022-04-04,Emailed Recommendations,"BCS,EC-HTOM,EC-AM,EC-KT,IRR,SNTC,SSPT,ST",FIELD,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364,106252,2161626,APJC,ASEAN_AREA,ASEAN_PH,ASEAN_COM_PHL,JG SUMMIT,<NA>,REACTIVE,Houman Asefi,JG2161626,2021-12-15,Complete,"BCS,BRW,DNA,IRR,PR,SNTC,SSPT,ST",FIELD,NaN,NaT
365,106253,4268448,APJC,ASEAN_AREA,ASEAN_PH,ASEAN_COM_PHL,METRO BANK,<NA>,REACTIVE,Houman Asefi,ME4268448,2021-12-15,Complete,"BCS,BRW,DNA,EC-AM,IRR,PR,SNTC,SSPT,ST",FIELD,NaN,NaT
366,106255,6827620,APJC,ASEAN_AREA,ASEAN_PH,ASEAN_COM_PHL,SECURITY BANK,<NA>,REACTIVE,Houman Asefi,SE6827620,2021-12-15,Complete,"BCS,BRW,DNA,EC-AM,IRR,PR,SNTC,SSPT,ST",FIELD,NaN,NaT
367,106256,4988524,APJC,ASEAN_AREA,ASEAN_PH,ASEAN_ENT_PHL,BDO,<NA>,REACTIVE,Houman Asefi,BD4988524,2021-12-15,Complete,"BCS,BRW,DNA,EC-AM,IRR,PR,SNTC,SSPT,ST",FIELD,NaN,NaT


In [164]:
def set_threshold_date(finbi_week_id):
    return must_be_prior_to_dict.get(finbi_week_id)

In [165]:
table['Threshold Date'] = table.apply(lambda x: set_threshold_date(x['Fiscal Week ID']), axis=1)

In [166]:
output.sort_values(by=['Compass GUID Name','Readout_Date'], inplace=True)

In [167]:
output.drop_duplicates(subset=['Compass GUID Name'], keep='last', inplace=True)

In [168]:
output

,Request ID,GUID,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,Compass GUID Name,Compass Deal ID,Campaign,Compass Advisor,Prehash_Entity,Readout_Date,Readout Delivery Status,Items Recommended,Campaign Type,CONCIERGE_ADVISOR_ASSIGNED,CONCIERGE_INTERNAL_READOUT_DT
360,106602,5883275,APJC,ANZ AREA,AU_COMMERCIAL,AU_COM_SA,ACCOLADE WINES AUSTRALIA LTD,<NA>,REACTIVE,Jayjay Towler,AC5883275,2022-02-09,Complete,"BCS,BRW,DNA,IRR,SNTC,SSPT,ST,SW/E-P",FIELD,NaN,NaT
29,107949,245627024,APJC,INDIA_AREA,IND_COMMERCIAL_OTH,IND_STH_KR_TS_AP,ADAMA INDIA PVT. LTD.,<NA>,REACTIVE,Houman Asefi,AD245627024,2021-12-09,Complete,"BCS,BRW,DNA,IRR,SNTC,ST",FIELD,NaN,NaT
188,18053,115996470,APJC,INDIA_AREA,IND_ENTERPRISE_MAJOR,IND_ENT_STRATEGIC,ADANI,<NA>,BCS - CLOSE THE GAP_2021Q4,Houman Asefi,AD115996470,2021-07-15,Complete,"BCS, BRW, DNA, IRR, PR, SNTC, SSPT, ST",TARGETING,NaN,NaT
34,108079,11556533,APJC,INDIA_AREA,IND_ENTERPRISE_MAJOR,IND_ENT_STRATEGIC,ADITYA BIRLA CAPITAL LTD,<NA>,REACTIVE,Houman Asefi,AD11556533,2022-01-19,Complete,"BCS,BRW,DNA,EC-AM,IRR,PR,SNTC,SSPT,ST",FIELD,NaN,NaT
244,21546,32552,APJC,JAPAN__,JP_ENTERPRISE,JP_ENT_FSI,AFLAC INC,55165765,BCS 3.0 TARGETING GLOBAL COMPASS IDENTIFIED RI...,Houman Asefi,AF32552,2021-12-06,Complete,"BCS,BRW,DNA,EC-AM,IRR,PR,SNTC,SSPT,ST",TARGETING,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,101280,146802,APJC,JAPAN__,JP_COMMERCIAL_OTH,JP_COM_SELECT,YAMAHA MOTOR CO. LTD.,43778140,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,Houman Asefi,YA146802,2021-11-22,Complete,"BCS,BRW,DNA,EC-AM,EC-HTOM,EC-KT,IRR,SNTC,SSPT,ST",TARGETING,NaN,NaT
118,101916,201036221,APJC,GREATER_CHINA,CN_NORTH,CN_ENT_NORTH_MET,YANTAI WANHUA GROUP,43697679,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,Houman Asefi,YA201036221,2021-11-24,Complete,"BCS,BRW,DNA,EC-AM,PR,IRR,SNTC,SSPT,ST",TARGETING,NaN,NaT
204,106584,111454259,APJC,INDIA_AREA,IND_ENTERPRISE_MAJOR,IND_ENT_BFSI_PVT,YES BANK,55164328,REACTIVE,Houman Asefi,YE111454259,2021-12-08,Complete,"BCS,BRW,DNA,EC-AM,IRR,PR,SNTC,SSPT,ST",FIELD,hasefi@cisco.com,2022-02-03
267,23518,3643901,APJC,JAPAN__,JP_ENTERPRISE,JP_ENT_MFG_1,YOKOGAWA ELECTRIC CORPORATION,55165750,BCS 3.0 TARGETING GLOBAL COMPASS IDENTIFIED RI...,Houman Asefi,YO3643901,2021-10-29,Complete,"BCS,BRW,EC-HTOM,EC-KT,IRR,SNTC,SSPT,ST",TARGETING,NaN,NaT


In [169]:
table['Sales Level 1'] = 'APJC'

In [170]:
compass_exists_table = pd.merge(table,output, how='left', left_on=['Prehash_Entity'], right_on = ['Prehash_Entity'])    # ,'CAV Name'       #,'Compass GUID Name'

In [171]:
compass_exists_table.shape

(517, 40)

In [172]:
compass_exists_table.rename(columns={'Sales Level 1_x':'Sales Level 1',
                                     'Sales Level 2_x':'Sales Level 2',
                                     'Sales Level 3_x':'Sales Level 3',
                                     'Sales Level 4_x':'Sales Level 4',
                                     'GUID_x':'GUID',
                                     'GUID_y':'Compass GUID'}, inplace=True)

compass_exists_table.drop(columns=(['Squeezed', 'Prepend','FakeID', 'Prehash_Entity','Sales Level 1_y','Sales Level 2_y', 'Sales Level 3_y','Sales Level 4_y',]), inplace=True)

In [173]:
compass_exists_table.columns

Index(['Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'Sales Level 4',
       'CAV ID', 'CAV Name', 'GUID', 'GUID Name', 'FINBI Deal ID',
       'FINBI Partner', 'AS Offer Type', 'CX Product Category',
       'Fiscal Quarter ID', 'Fiscal Week ID', 'ReportWeek', 'Service Source',
       'ACV', 'TCV', 'Ind', 'Threshold Date', 'Request ID', 'Compass GUID',
       'Compass GUID Name', 'Compass Deal ID', 'Campaign', 'Compass Advisor',
       'Readout_Date', 'Readout Delivery Status', 'Items Recommended',
       'Campaign Type', 'CONCIERGE_ADVISOR_ASSIGNED',
       'CONCIERGE_INTERNAL_READOUT_DT'],
      dtype='object')

In [174]:
compass_exists_table

,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,CAV ID,CAV Name,GUID,GUID Name,FINBI Deal ID,FINBI Partner,...,Compass GUID Name,Compass Deal ID,Campaign,Compass Advisor,Readout_Date,Readout Delivery Status,Items Recommended,Campaign Type,CONCIERGE_ADVISOR_ASSIGNED,CONCIERGE_INTERNAL_READOUT_DT
0,APJC,ANZ AREA,ANZ_CONTRA,ANZ_CONTRA_OPS,100665,INSURANCE AUSTRALIA GROUP LTD,5883288,INSURANCE AUSTRALIA GROUP LTD,56027888,UNKNOWN,...,INSURANCE AUSTRALIA GROUP LTD,<NA>,REACTIVE,Jayjay Towler,2021-12-09,Complete,"BCS,BRW,DNA,EC-HTOM,IRR,SNTC,SSPT,ST",FIELD,NaN,NaT
1,APJC,ANZ AREA,ANZ_CONTRA,ANZ_CONTRA_OPS,100675,BHP BILLITON LTD,156215,BHP BILLITON LTD,-9999,UNKNOWN,...,BHP BILLITON IRON ORE,20908483,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,Houman Asefi,2021-11-30,Complete,"BCS,BRW,DNA,EC-AM,EC-HTOM,IRR,PR,SSPT,ST",TARGETING,NaN,NaT
2,APJC,ANZ AREA,ANZ_CONTRA,ANZ_CONTRA_OPS,100675,BHP BILLITON LTD,156215,BHP BILLITON LTD,-9999,UNKNOWN,...,BHP BILLITON LTD,55520762,SUCCESS TRACKS DYNAMIC CAMPAIGN,Jayjay Towler,2022-02-08,Complete,"SNTC,ST",TARGETING,NaN,NaT
3,APJC,ANZ AREA,ANZ_CONTRA,ANZ_CONTRA_OPS,103142,JONES LANG LASALLE INC,21175,JONES LANG LASALLE INC,53987660,UNKNOWN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,APJC,ANZ AREA,ANZ_CONTRA,ANZ_CONTRA_OPS,107865,SWINBURNE UNIVERSITY OF TECHNOLOGY,6163312,SWINBURNE UNIVERSITY OF TECHNOLOGY,-9999,UNKNOWN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,APJC,ROK_AREA,KOR_PUBLIC SECTOR,KR_PS_CENTRAL,1809767,KOREA AIRPORT CORPORATION,4445502,GOVERNMENT OF THE REPUBLIC OF KOREA,20228141,SNET SYSTEMS INC.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
513,APJC,ROK_AREA,KOR_SMALL,KR_SMALL_SBU,640947,THE DAEGU BANK LTD,4543414,THE DAEGU BANK LTD,44228135,"Sangsangin Co., Ltd.",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
514,APJC,ROK_AREA,KOR_SMALL,KR_SMALL_SBU,2550366,TELECHIPS INC.,271853584,TELECHIPS INC.,55270165,UNKNOWN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
515,APJC,ROK_AREA,KOR_SMALL,KR_SMALL_SBU,2550366,TELECHIPS INC.,271853584,TELECHIPS INC.,55270165,INSUNG INFORMATION CO LTD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [175]:
# compass_exists_table = pd.merge(compass_exists_table, services_source, how='left', left_on = 'FINBI Deal ID', right_on = 'Deal ID')

In [176]:
compass_exists_table['Readout_Date'] = pd.to_datetime(compass_exists_table['Readout_Date'])

In [177]:
compass_exists_table['Threshold Date'] = pd.to_datetime(compass_exists_table['Threshold Date'])

In [178]:
condition1 = (compass_exists_table['Readout_Date'] <= compass_exists_table['Threshold Date'])  #
# condition2 = ((compass_exists_table['Readout_Date'].isnull()) & (compass_exists_table['Deal ID']==-9999))

In [179]:
compass_exists_table = compass_exists_table.loc[condition1] #[['FINBI Deal ID','Threshold Date','Readout_Date', 'Compass Deal ID', 'Deal ID']]

In [180]:
# compass_exists_table = compass_exists_table.loc[condition1 & ~condition2] #[['FINBI Deal ID','Threshold Date','Readout_Date', 'Compass Deal ID', 'Deal ID']]

In [181]:
compass_exists_table

,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,CAV ID,CAV Name,GUID,GUID Name,FINBI Deal ID,FINBI Partner,...,Compass GUID Name,Compass Deal ID,Campaign,Compass Advisor,Readout_Date,Readout Delivery Status,Items Recommended,Campaign Type,CONCIERGE_ADVISOR_ASSIGNED,CONCIERGE_INTERNAL_READOUT_DT
0,APJC,ANZ AREA,ANZ_CONTRA,ANZ_CONTRA_OPS,100665,INSURANCE AUSTRALIA GROUP LTD,5883288,INSURANCE AUSTRALIA GROUP LTD,56027888,UNKNOWN,...,INSURANCE AUSTRALIA GROUP LTD,<NA>,REACTIVE,Jayjay Towler,2021-12-09,Complete,"BCS,BRW,DNA,EC-HTOM,IRR,SNTC,SSPT,ST",FIELD,NaN,NaT
1,APJC,ANZ AREA,ANZ_CONTRA,ANZ_CONTRA_OPS,100675,BHP BILLITON LTD,156215,BHP BILLITON LTD,-9999,UNKNOWN,...,BHP BILLITON IRON ORE,20908483,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,Houman Asefi,2021-11-30,Complete,"BCS,BRW,DNA,EC-AM,EC-HTOM,IRR,PR,SSPT,ST",TARGETING,NaN,NaT
2,APJC,ANZ AREA,ANZ_CONTRA,ANZ_CONTRA_OPS,100675,BHP BILLITON LTD,156215,BHP BILLITON LTD,-9999,UNKNOWN,...,BHP BILLITON LTD,55520762,SUCCESS TRACKS DYNAMIC CAMPAIGN,Jayjay Towler,2022-02-08,Complete,"SNTC,ST",TARGETING,NaN,NaT
8,APJC,ANZ AREA,ANZ_CONTRA,ANZ_CONTRA_OPS,171655,AUSTRALIAN ENERGY MARKET OPERATOR LTD,3427559,AUSTRALIAN ENERGY MARKET OPERATOR LTD,44195553,UNKNOWN,...,AUSTRALIAN ENERGY MARKET OPERATOR LTD,55171696,BCS 3.0 TARGETING GLOBAL CXPM HIGH PROPENSITY ...,Houman Asefi,2021-10-15,Complete,"BCS,DNA,EC-HTOM,IRR,SNTC,ST,SSPT,BRW",TARGETING,NaN,NaT
11,APJC,ANZ AREA,ANZ_CONTRA,ANZ_CONTRA_OPS,299621,QUEENSLAND STATE GOVERNMENT,197704874,QUEENSLAND STATE GOVERNMENT,-9999,UNKNOWN,...,QUEENSLAND HEALTH,43527165,REACTIVE - CXIB,Houman Asefi,2021-07-21,Complete,"BCS, BRW, DNA, EC-HTOM, EC-KT, IRR, SNTC, SSPT...",CXIB,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
470,APJC,JAPAN__,JP_ENTERPRISE_CGEM,GLL_TOYOTA,498427,DENSO CORPORATION,147125,DENSO CORPORATION,43980869,UNIADEX LTD,...,DENSO CORPORATION,55520071,SUCCESS TRACKS DYNAMIC CAMPAIGN,Houman Asefi,2022-02-08,Complete,"BCS,BRW,DNA,EC-AM,IRR,PR,SNTC,SSPT,ST",TARGETING,NaN,NaT
471,APJC,JAPAN__,JP_ENTERPRISE_CGEM,GLL_TOYOTA,498427,DENSO CORPORATION,147125,DENSO CORPORATION,43980869,UNIADEX LTD,...,DENSO CORPORATION,55520071,SUCCESS TRACKS DYNAMIC CAMPAIGN,Houman Asefi,2022-02-08,Complete,"BCS,BRW,DNA,EC-AM,IRR,PR,SNTC,SSPT,ST",TARGETING,NaN,NaT
496,APJC,ROK_AREA,KOREA_MISC,KOREA_MISC_OPS,669694,HYUNDAI MOTOR COMPANY,146484,HYUNDAI MOTOR COMPANY,44179289,UNKNOWN,...,HYUNDAI AUTOEVER,44144531,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,Houman Asefi,2021-11-15,Complete,"BCS,DNA,IRR,SNTC,SSPT,ST",TARGETING,NaN,NaT
497,APJC,ROK_AREA,KOREA_MISC,KOREA_MISC_OPS,669694,HYUNDAI MOTOR COMPANY,146484,HYUNDAI MOTOR COMPANY,44236372,UNKNOWN,...,HYUNDAI AUTOEVER,44144531,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,Houman Asefi,2021-11-15,Complete,"BCS,DNA,IRR,SNTC,SSPT,ST",TARGETING,NaN,NaT


In [182]:
# compass_exists_table = compass_exists_table[['Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'CAV ID', 'CAV Name',
#        'GUID', 'GUID Name', 'Deal ID', 'CX Product Category',
#        'Fiscal Quarter ID', 'Fiscal Week ID', 'ReportWeek', 'AS Offer Type', 'TCV', 'ACV', 'Ind',
#        'Request ID', 'Readout_Date','Compass GUID Name','Compass Exists?','Compass Claim?', 'Notes']]

In [183]:
product_types = {'BCS 2.x':'BCS 2',
                 'BCS 2.0':'BCS 2',
                 'BCS 3.x':'BCS 3',
                 'BCS 3.0':'BCS 3',
                 'HTEC':'HTEC',
                 'Expert Care':'HTEC',
                 'Level 1':'ST',
                 'Level 2':'ST',
                 'Solution Support':'SSPT',
                 'SuccessTracks':'ST',
                 'Campus Network':'ST',
                 'SNTC':'SNTC',
                 'Unknown':'UNK'}

In [184]:
# compass_exists_table.rename(columns={'Deal ID':'FINBI Deal ID'}, inplace=True)

In [185]:
compass_exists_table.rename(columns={'AS Offer Type':'Product Type'}, inplace=True)

C:\Users\phsheari\.conda\envs\compass_env\lib\site-packages\pandas\core\frame.py:4438: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [186]:
compass_exists_table['Compass Deal ID'] = compass_exists_table['Compass Deal ID'].apply(convert_nan_to_int)

<ipython-input-186-e2b0b5ca6b42>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compass_exists_table['Compass Deal ID'] = compass_exists_table['Compass Deal ID'].apply(convert_nan_to_int)


In [187]:
compass_exists_table['Product Type'] = compass_exists_table['CX Product Category'].apply(lambda x: product_types.get(x))

<ipython-input-187-dcc40366fa47>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compass_exists_table['Product Type'] = compass_exists_table['CX Product Category'].apply(lambda x: product_types.get(x))


In [188]:
compass_exists_table.sort_values(by=['CAV Name', 'GUID Name', 'FINBI Deal ID', 'Fiscal Quarter ID', 'ReportWeek', 'Sales Level 1','Sales Level 2','Sales Level 3', 'Sales Level 4','Product Type','TCV'], inplace=True)

<ipython-input-188-12d2f74770a7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compass_exists_table.sort_values(by=['CAV Name', 'GUID Name', 'FINBI Deal ID', 'Fiscal Quarter ID', 'ReportWeek', 'Sales Level 1','Sales Level 2','Sales Level 3', 'Sales Level 4','Product Type','TCV'], inplace=True)


In [189]:
def apply_targeting_campaign_flag(compass_campaign_name):
    if pd.notnull(compass_campaign_name) and ('target' in compass_campaign_name.lower() or 'bcs' in compass_campaign_name.lower() or 'major' in compass_campaign_name.lower()  or 'success' in compass_campaign_name.lower()):
        return True
    else:
        return False

In [190]:
compass_exists_table['Target Campaign Flag'] = compass_exists_table['Campaign'].apply(lambda x: apply_targeting_campaign_flag(x))

<ipython-input-190-f69892c49de5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compass_exists_table['Target Campaign Flag'] = compass_exists_table['Campaign'].apply(lambda x: apply_targeting_campaign_flag(x))


In [191]:
compass_exists_table['DupeCheck'] =  compass_exists_table.duplicated(['Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'CAV ID', 'CAV Name','GUID', 'GUID Name', 'FINBI Partner', 'FINBI Deal ID','CX Product Category', 'Fiscal Quarter ID', 'Fiscal Week ID','ReportWeek', 'Product Type', 'TCV'], keep=False)

<ipython-input-191-c38f311cd3b0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compass_exists_table['DupeCheck'] =  compass_exists_table.duplicated(['Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'CAV ID', 'CAV Name','GUID', 'GUID Name', 'FINBI Partner', 'FINBI Deal ID','CX Product Category', 'Fiscal Quarter ID', 'Fiscal Week ID','ReportWeek', 'Product Type', 'TCV'], keep=False)


In [192]:
compass_exists_table['Compass Exists?'] = ''
compass_exists_table['Compass Claim?'] = ''
compass_exists_table['Notes'] = ''

<ipython-input-192-0c0e975f3ae4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compass_exists_table['Compass Exists?'] = ''
<ipython-input-192-0c0e975f3ae4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compass_exists_table['Compass Claim?'] = ''
<ipython-input-192-0c0e975f3ae4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

In [193]:
compass_exists_table = compass_exists_table[['Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'Sales Level 4','CAV ID', 'CAV Name', 'GUID', 'GUID Name', 'FINBI Partner', 'FINBI Deal ID',
                                             'CX Product Category', 'Fiscal Quarter ID', 'Fiscal Week ID', 'ReportWeek', 'Product Type', 'TCV', 'ACV', 'Ind', 'DupeCheck', 
                                             'Request ID', 'Compass GUID','Compass GUID Name', 'Compass Deal ID', 'Campaign', 'Threshold Date', 'Readout_Date',
                                             'Compass Advisor', 'Readout Delivery Status', 'Items Recommended', 'Campaign Type', 'Service Source', 'CONCIERGE_ADVISOR_ASSIGNED', 'CONCIERGE_INTERNAL_READOUT_DT','Target Campaign Flag', 'Compass Exists?', 
                                             'Compass Claim?', 'Notes', 
                                            ]]

In [194]:
# compass_exists_table = compass_exists_table.loc[(compass_exists_table['Readout Delivery Status']!= 'Declined')] #(compass_exists_table['Service Source'].notnull()) | 

In [195]:
# compass_exists_table = compass_exists_table.loc[(compass_exists_table['Service Source'].notnull())]

In [196]:
table.rename(columns={'AS Offer Type':'Product Type'}, inplace=True)

In [197]:
table['Product Type'] = table['CX Product Category'].apply(lambda x: product_types.get(x))

In [198]:
# success.loc[(success['Fiscal Quarter ID']=='2021Q2') & (success['Fiscal Week ID'] >= 2021053)]

In [199]:
#compass_exists_table = compass_exists_table.loc[compass_exists_table['Request ID'].notnull()]

In [200]:
compass_exists_table

,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,CAV ID,CAV Name,GUID,GUID Name,FINBI Partner,FINBI Deal ID,...,Readout Delivery Status,Items Recommended,Campaign Type,Service Source,CONCIERGE_ADVISOR_ASSIGNED,CONCIERGE_INTERNAL_READOUT_DT,Target Campaign Flag,Compass Exists?,Compass Claim?,Notes
84,APJC,APJ_SP,APJ_SP_ANZ,AU_SP_MJ_AUSSIEBB_SC,182407,AUSSIE BROADBAND PTY LTD,144363794,AUSSIE BROADBAND PTY LTD,AUSSIE BROADBAND LIMITED,55940311,...,Complete,"BCS,DNA,EC-HTOM,IRR,SNTC,SSPT,ST,BRW",TARGETING,NEW,NaN,NaT,True,,,
59,APJC,ANZ AREA,AU_PUBLIC SECTOR,AU_DEFENCE REGION,2099659,AUSTRALIAN DEFENCE AND SECURITY-CIOG,4104479,COMMONWEALTH GOVERNMENT,DATA#3 LIMITED,42711511,...,Complete,"BCS,DNA,IRR,PR,SNTC,SSPT,ST",FIELD,NEW,NaN,NaT,False,,,
60,APJC,ANZ AREA,AU_PUBLIC SECTOR,AU_DEFENCE REGION,2099659,AUSTRALIAN DEFENCE AND SECURITY-CIOG,4104479,COMMONWEALTH GOVERNMENT,DXC CONNECT PTY LTD,55492838,...,Complete,"BCS,DNA,IRR,PR,SNTC,SSPT,ST",FIELD,NEW,NaN,NaT,False,,,
53,APJC,ANZ AREA,AU_PUBLIC SECTOR,AU_DEFENCE REGION,1807909,AUSTRALIAN DEPARTMENT OF DEFENCE,4104479,COMMONWEALTH GOVERNMENT,UNKNOWN,44335240,...,Complete,"BCS,DNA,IRR,PR,SNTC,SSPT,ST",FIELD,RENEW,NaN,NaT,False,,,
54,APJC,ANZ AREA,AU_PUBLIC SECTOR,AU_DEFENCE REGION,1807909,AUSTRALIAN DEPARTMENT OF DEFENCE,4104479,COMMONWEALTH GOVERNMENT,DATA#3 LIMITED,54001783,...,Complete,"BCS,DNA,IRR,PR,SNTC,SSPT,ST",FIELD,NEW,NaN,NaT,False,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348,APJC,INDIA_AREA,IND_ENTERPRISE_ITES,GLL_WIPRO,103143,WIPRO LTD,2159580,WIPRO LTD,PROACTIVE DATA SYSTEMS PVT LTD,56039449,...,Complete,"BCS, BRW, DNA, EC-HTOM, IRR, PR, SNTC, SSPT, ST",TARGETING,NEW,NaN,NaT,True,,,
353,APJC,INDIA_AREA,IND_ENTERPRISE_ITES,GLL_WIPRO,103143,WIPRO LTD,2159580,WIPRO LTD,PROACTIVE DATA SYSTEMS PVT LTD,56039555,...,Complete,"BCS,BRW,DNA,EC-AM,EC-HTOM,EC-KT,IRR,PR,SNTC,SS...",TARGETING,UNKNOWN,NaN,NaT,True,,,
354,APJC,INDIA_AREA,IND_ENTERPRISE_ITES,GLL_WIPRO,103143,WIPRO LTD,2159580,WIPRO LTD,PROACTIVE DATA SYSTEMS PVT LTD,56039555,...,Complete,"BCS, BRW, DNA, EC-HTOM, IRR, PR, SNTC, SSPT, ST",TARGETING,UNKNOWN,NaN,NaT,True,,,
351,APJC,INDIA_AREA,IND_ENTERPRISE_ITES,GLL_WIPRO,103143,WIPRO LTD,2159580,WIPRO LTD,PROACTIVE DATA SYSTEMS PVT LTD,56039555,...,Complete,"BCS,BRW,DNA,EC-AM,EC-HTOM,EC-KT,IRR,PR,SNTC,SS...",TARGETING,NEW,NaN,NaT,True,,,


In [201]:
## Check the Targeting Campaign Flag Attribution
compass_exists_table[['Campaign','Target Campaign Flag']].drop_duplicates(keep='first')

,Campaign,Target Campaign Flag
84,BCS 3.0 TARGETING GLOBAL COMPASS IDENTIFIED RI...,True
59,REACTIVE,False
8,BCS 3.0 TARGETING GLOBAL CXPM HIGH PROPENSITY ...,True
215,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,True
2,SUCCESS TRACKS DYNAMIC CAMPAIGN,True
404,BCS - CLOSE THE GAP_2021Q4,True
11,REACTIVE - CXIB,False
117,TELEMETRY,False


In [202]:
with pd.ExcelWriter(fin_path + "\\" + endfilename) as writer:  #endfilename  endfilename
    df.to_excel(writer, sheet_name = 'RawData', index=False)
    table.to_excel(writer, sheet_name = 'Pivot', index = False)
    compass_exists_table.to_excel(writer, sheet_name = "Compass Exists", index=False)